In [3]:
from pulp import *
import time
import numpy as np
from random import randint


#need to figure out how to create 30 insances and loop across all of them

prob = pulp.LpProblem("Transportation Problem", pulp.LpMinimize)

#Supply locations, X, from 1 to N
N = 20
Supply = {}
for s in range (0,N):
    Supply[s] = randint(1,100)

print "SUPPLY"
print(Supply)


#demand locations, Y from 1 to M
#last demand node uses if statement to ensure it does not exceed overall supply totals
#change supply and demand.  add supply, subtract from demand, fill last supply node + 1 only for non int
M = 20
Demand = {}
for d in range (0,M-1):
    Demand[d] = randint(1,100)
print "DEMAND -1"
print(Demand)
Supply_total = sum(Supply.values())
print"shitty supply total"
print (Supply_total)
Demand_total = sum(Demand.values())
print"shitty demand total 1"
print (Demand_total)
FIX = sum(Supply.values())-sum(Demand.values())
print "FIX"
print (FIX)
if FIX >= 0:
    Demand[M-1] = FIX
else:
    Demand[M-1] = 0

print "FIXED DEMAND"
print(Demand)
print "shitty new demand total"
Demand_total2 = sum(Demand.values())
print (Demand_total2)

#create matrix with arc weights
ARC = np.random.uniform(1,9,size=(N,M))
print "ARC weights"
print (ARC)

#create ARC connections between supply and demand nodes
#This should look like A1, A2...where A is supply node and # is demand node
#create dictionary to store variables so we can later assigned values to them
Routes = {}
Routes = [(n,m) for n in range(0,N) for m in range(0,M)]


#check out zip within python - will go across two data types that are iterable and the same length

print "ROUTES"
print (Routes)

print "BREAK _________________"
Route_vars = LpVariable.dicts("X",(Routes),0,None,LpInteger)
print "ROUTE VARS"
print (Route_vars)
print "BREAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAk"

#test code to make sure that variables were input and reference correctly and then correctly multiplied by arc weights
#iter = 0
#for (n,m) in Route_vars:
#    test = ARC[n][m] * Route_vars[n,m]
#test = ARC[1][1] * Route_vars[1,1]
#    print"break-------------"
#    print"test"
#    print (test)
#    print (iter)
#    iter = iter +1

# The objective function that we want to minimize: the total cost
# MIN SUM 1:M SUM 1:N Cij * Xij
#this function loops across the Route_vars dictionary and multiplies the corresponding ARC weight against it
for (n,m) in Route_vars:
    prob += lpSum(ARC[n][m] * Route_vars[n,m]for d in Demand)


# supply constraints - non neg inherent to random numbers generated
#supply must be equal to demand
#not sure the next step but close...

#test sum of route vars for demand
for s in Supply:
    Test_sum = sum(Route_vars[s,m] for d in Demand)
    print (Test_sum)
    print "tiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiits"

for s in Supply:
    prob += lpSum([Route_vars[s,m] for d in Demand]) <= Supply[s]

#test sum of route vars for demand
for d in Demand:
    Test_sum = sum(Route_vars[n,d] for s in Supply)
    print (Test_sum)
    print "suck ittttttttttttttttttttttttttttttttttt"

# demand constraints - non neg inherent to random numbers generated
#demand must be equal to supply
for d in Demand:
    prob += lpSum([Route_vars[n,d] for s in Supply]) >= Demand[d]





name = {1:"GLPK",2:"CPLEX",3:"GUROBI",4:"CBC"}

#CBC   
start_time = time.time()
prob.solve()
print name[4]
    #print status
print("Status:", LpStatus[prob.status])
    #print time
CBCtime = (time.time() - start_time)
print("--- %s seconds ---" % CBCtime)
    # Print out the results
for v in prob.variables():
    print v.name, "=", v.varValue
print "-----------break-----------"

#GUROBI
start_time = time.time()
prob.solve(GUROBI(msg=0))
print name[3]
    #print status
print("Status:", LpStatus[prob.status])
    #print time
GUROBItime = (time.time() - start_time)
print("--- %s seconds ---" % GUROBItime)
    # Print out the results
for v in prob.variables():
    print v.name, "=", v.varValue
print "-----------break-----------"

#GLPK
start_time = time.time()
prob.solve(GLPK(msg=0))
print name[1]
    #print status
print("Status:", LpStatus[prob.status])
    #print time
GLPKtime = (time.time() - start_time)
print("--- %s seconds ---" % GLPKtime)
    # Print out the results
for v in prob.variables():
    print v.name, "=", v.varValue
print "-----------break-----------"


start_time = time.time()
prob.solve(CPLEX(msg=0))
print name[2]
    #print status
print("Status:", LpStatus[prob.status])
    #print time
CPLEXtime = (time.time() - start_time)
print("--- %s seconds ---" % CPLEXtime)
    # Print out the results
for v in prob.variables():
    print v.name, "=", v.varValue
print "-----------break-----------"


SUPPLY
{0: 80, 1: 21, 2: 77, 3: 83, 4: 43, 5: 97, 6: 66, 7: 58, 8: 8, 9: 29, 10: 10, 11: 99, 12: 88, 13: 94, 14: 40, 15: 5, 16: 97, 17: 25, 18: 75, 19: 29}
DEMAND -1
{0: 47, 1: 60, 2: 69, 3: 94, 4: 60, 5: 62, 6: 1, 7: 2, 8: 66, 9: 68, 10: 29, 11: 33, 12: 9, 13: 39, 14: 59, 15: 28, 16: 31, 17: 42, 18: 76}
shitty supply total
1124
shitty demand total 1
875
FIX
249
FIXED DEMAND
{0: 47, 1: 60, 2: 69, 3: 94, 4: 60, 5: 62, 6: 1, 7: 2, 8: 66, 9: 68, 10: 29, 11: 33, 12: 9, 13: 39, 14: 59, 15: 28, 16: 31, 17: 42, 18: 76, 19: 249}
shitty new demand total
1124
ARC weights
[[ 4.44562376  8.38243698  8.59509851  8.93813292  1.5348189   5.810107
   5.05216228  1.8781127   1.12810406  1.16687891  7.37192156  7.5412301
   4.64827755  5.59028805  3.07827629  7.07146195  8.3115609   3.87644535
   6.95305322  4.39752314]
 [ 7.49565811  2.61597613  5.90160996  2.87745707  5.43080034  3.4735894
   2.04981729  5.89538621  6.95335455  4.21289083  3.3797851   3.6329713
   7.91455258  7.62132081  8.9026468   7